In [1]:
import pandas as pd
from string import digits

In [2]:
matches_df = pd.read_csv("data/espn_data_240618.csv")

In [3]:
matches_df.head()

,Date,Team1,Team1_score,Team2,Team2_score,team1_fouls,team2_fouls,team1_yellow_cards,team2_yellow_cards,team1_red_cards,...,team1_shots,team2_shots,team1_avg_wins,team1_avg_draws,team1_avg_gd,team2_avg_wins,team2_avg_draws,team2_avg_gd,team1_lineup,team2_lineup
0,20140612,Brazil,3,Croatia,1,5,21,2,2,0,...,14 (9),10 (4),1.0,0.0,3.2,0.6,0.4,0.8,12 Júlio César_4 David Luiz_3 Thiago Silva_6 M...,1 Stipe Pletikosa_6 Dejan Lovren _5 Vedran Co...
1,20140613,Mexico,1,Cameroon,0,11,12,1,1,0,...,9 (5),10 (4),0.4,0.2,0.6,0.4,0.2,-0.4,13 Guillermo Ochoa_4 Rafael Márquez_15 Héctor ...,16 Charles Itandje_14 Aurélien Chedjou_3 Nicol...
2,20140613,Spain,1,Netherlands,5,5,18,1,3,0,...,9 (6),13 (11),0.8,0.0,1.0,0.4,0.4,0.2,1 Iker Casillas _15 Sergio Ramos_3 Gerard Piq...,1 Jasper Cillessen_2 Ron Vlaar_4 Bruno Martins...
3,20140613,Chile,3,Australia,1,9,18,1,3,0,...,11 (9),13 (6),0.6,0.0,1.0,0.4,0.2,0.4,1 Claudio Bravo_18 Gonzalo Jara_17 Gary Medel_...,1 Mathew Ryan_6 Matthew Spiranovic_22 Alex Wil...
4,20140614,Colombia,3,Greece,0,19,14,1,2,0,...,12 (8),12 (7),0.4,0.6,1.0,0.2,0.6,-0.2,1 David Ospina_3 Mario Yepes_2 Cristián Zapata...,1 Orestis Karnezis_19 Sokratis Papastathopoulo...


Create features from the line up and form to predict the goal difference

In [4]:
def get_players_from_team_string(team_string):
    remove_digits = str.maketrans('', '', digits)
    return list(map(lambda x: x.translate(remove_digits).replace("  ","").replace("--","").replace("\n","")[1:], team_string.split("_")))

In [5]:
year = "2014"

In [6]:
matches_df["year"] = matches_df.Date.apply(lambda x: str(x)[:4])

In [7]:
year_matches_df = matches_df[matches_df.year == year]

### Player stats

First, process each row and get full list of players to search for.

In [10]:
# retrieve player database
year_player_stats_df = pd.read_csv("data/to_scrape/scraped_"+year+".csv")

In [23]:
def get_team_stats(team_string, year_player_stats_df):
    player_list = get_players_from_team_string(team_string)
    lineup_stats_df = year_player_stats_df[year_player_stats_df.name.isin(player_list)]
    lineup_stats_df = lineup_stats_df[~lineup_stats_df.appearances.isnull()]
    lineup_stats_df = lineup_stats_df.replace("-",0)
    if lineup_stats_df.shape[0] == 0:
        return [0,0,0,0,0]
    avg_age = lineup_stats_df.age.apply(int).mean()
    avg_gpa = (lineup_stats_df.goals.apply(int) / lineup_stats_df.appearances.apply(int)).mean()
    avg_apa = (lineup_stats_df.assists.apply(int) / lineup_stats_df.appearances.apply(int)).mean()
    avg_ycpa = (lineup_stats_df.yellow_cards.apply(int) / lineup_stats_df.appearances.apply(int)).mean()
    total_minutes = lineup_stats_df.minutes.apply(int).sum()
    return [avg_age, avg_gpa, avg_apa, avg_ycpa, total_minutes]

In [45]:
def format_age(age_string):
    if len(str(age_string)) == 2:
        return age_string
    return age_string.split("(")[1].split(")")[0]

In [12]:
year_player_stats_df.age = year_player_stats_df.age.apply(format_age)

In [20]:
year_matches_df = year_matches_df[~year_matches_df.team1_lineup.isnull()]
year_matches_df = year_matches_df[~year_matches_df.team2_lineup.isnull()]

In [27]:
year_matches_team1_stats = year_matches_df.team1_lineup.apply(lambda x: get_team_stats(x, year_player_stats_df))
year_matches_team2_stats = year_matches_df.team2_lineup.apply(lambda x: get_team_stats(x, year_player_stats_df))

In [30]:
year_matches_team1_df = year_matches_team1_stats.apply(lambda x:pd.Series(x))
year_matches_team2_df = year_matches_team2_stats.apply(lambda x:pd.Series(x))

In [33]:
year_matches_team1_df.columns = ["avg_age_t1", "avg_gpa_t1", "avg_apa_t1", "avg_ycpa_t1", "total_minutes_t1"]
year_matches_team2_df.columns = ["avg_age_t2", "avg_gpa_t2", "avg_apa_t2", "avg_ycpa_t2", "total_minutes_t2"]

In [37]:
year_matches_df_final = pd.concat([year_matches_df, year_matches_team1_df, year_matches_team2_df],axis = 1)

In [46]:
world_cup_years = ["2014","2010","2006","2002","1998","1994"]

for year in world_cup_years:
    year_matches_df = matches_df[matches_df.year == year]
    year_player_stats_df = pd.read_csv("data/to_scrape/scraped_"+year+".csv")
    year_player_stats_df.age = year_player_stats_df.age.apply(format_age)
    year_matches_df = year_matches_df[~year_matches_df.team1_lineup.isnull()]
    year_matches_df = year_matches_df[~year_matches_df.team2_lineup.isnull()]
    year_matches_team1_stats = year_matches_df.team1_lineup.apply(lambda x: get_team_stats(x, year_player_stats_df))
    year_matches_team2_stats = year_matches_df.team2_lineup.apply(lambda x: get_team_stats(x, year_player_stats_df))
    year_matches_team1_df = year_matches_team1_stats.apply(lambda x:pd.Series(x))
    year_matches_team2_df = year_matches_team2_stats.apply(lambda x:pd.Series(x))
    year_matches_team1_df.columns = ["avg_age_t1", "avg_gpa_t1", "avg_apa_t1", "avg_ycpa_t1", "total_minutes_t1"]
    year_matches_team2_df.columns = ["avg_age_t2", "avg_gpa_t2", "avg_apa_t2", "avg_ycpa_t2", "total_minutes_t2"]
    year_matches_df_final = pd.concat([year_matches_df, year_matches_team1_df, year_matches_team2_df],axis = 1)
    year_matches_df_final.to_csv("data/final/full_" + year + '.csv', index=False)